## TalkingData AdTracking Fraud Detection Challenge 


このノートブックでは [TalkingDataの不正検知コンペ](https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection) を題材にして
アプリ広告がクリックがされたダウンロード。

今回のノートブックの目的は
- データ分析からモデリングまでの一連の流れを体験する。
- コンペの評価基準(AUC)で検証を行う。

になります。

スコアは二の次なので、
ハイスコアを求める方は他のノートブックを参照ください。

ノートブックは下記の流れで進めます。

1. 問題定義
1. データを用意する
1. 前処理
1. データの分析
1. モデル作成
1. 可視化して報告する。


## 1.問題定義

不正クリックかどうかを検知するのに、なぜ広告がクリックされた後アプリがダウンロードされるかを調べる必要があるのでしょうか？

まずドメイン知識として、「なぜ不正クリックをするのか」を調べました。

不正クリックを行って利益を得ようとする人は大きく「競合アプリの開発者」と「広告ネットワーク会社」に分かれます。

- 競合アプリの開発者
    - 広告出稿主は広告費として一日の上限金額を設定しています。１クリックごとに金額が発生するため、一定回数以上クリックされるとその広告は表示されなくなります。これを利用して競合アプリの広告を表示させなくすることで自分たちのアプリの広告を表示させようとしています。
- 広告ネットワーク会社
    - 広告ネットワーク会社は広告出稿主からクリックごとに料金をもらえます。そのため不正クリックでクリックを水増しすることで利益を得ようとします。

概要でも説明がありましたが、不正業者はなるべく多くのスマートフォンを使って膨大なクリックを発生させたいので、広告をクリックしてもアプリをダウンロードすることは決してありません。

つまり広告をクリックしたあとアプリをダウンロードされないクリックは不正クリックとみなします。

今回のコンペティションの目的は「**広告がクリックされたあとにダウンロードされるかされないか」を精度良く予測すること**です。

## 2.データを用意する

今回はダウンロードした時間にあたる項目'attributed_time'はテストデータにないため、
これを除いてデータを読み込みます。

In [1]:
# 必要なモジュールのインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
%matplotlib inline
import gc

import lightgbm as lgb

import warnings
warnings.filterwarnings('ignore')

In [2]:
from subprocess import check_output # python上でunixコマンドを使うライブラリ
print(check_output(['ls', '../input']).decode('utf-8')) # lsして文字列をutf-8で返す

Iris.csv
test.csv.zip
train.csv
train.csv.zip
train_sample.csv.zip



train.csv.zipを見ると1.3GBあるので、メモリにロードできるか確認していきます。
まずipアドレスだけを確認します。

In [3]:
data_path = '../input/'
# read the column
column = 'ip'
column_df = pd.read_csv(data_path+ 'train.csv.zip', usecols=[column])
# If memory is a big problem for you, please use the following command
# column_df = pd.read_csv(file_path, nrows=20000000, usecols=[column])
# Display memory usage in GB
print("Memory usage = %.3f GB" % (column_df.memory_usage().sum() / 1024 ** 3))
print("row count: {}".format(column_df.count()))
# Find the min 
the_min = column_df[column].min()
# Find the max
the_max = column_df[column].max()
# display min and max and determine the minimum data type
print("min=", the_min, ", max=", the_max)

Memory usage = 1.378 GB
row count: ip    184903890
dtype: int64
min= 1 , max= 364778


このように1.8億件のデータがあり、１項目だけでメモリが1.378GB使います。

残念なことにpandasは数値項目はint64を確保してしまいます。
そのため上記のように各項目に対して最大最小値を確認して、最小の型を選択する必要があります。

今回は時間制約もあり、学習データは10万件のサンプルデータで進めます。


In [4]:
data_path = '../input/'
dtypes = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'is_attributed' : 'uint8',
        'click_id'      : 'uint32'
        }

train_df = pd.read_csv(data_path + 'train_sample.csv.zip', 
                       # nrows would select the first rows not the last rows so you need to use skiprows parameter
                        # nrows=45000000, 45e6
                                # We use skiprows here with the range function, a lambda function call would be possible but looks pretty slower
                                        #skiprows=range(1, int(184903890 - 10000000)),
                                        dtype=dtypes, 
                                        usecols=['ip','app','device','os', 'channel', 'click_time', 'is_attributed'])

test_df = pd.read_csv(data_path + 'test.csv.zip', 
                      dtype=dtypes, 
                      usecols=['ip','app','device','os', 'channel', 'click_time', 'click_id'])


## 3.前処理

ダウンロードされた時間はテストデータにないので除外してデータを確認します。
欠損値がないか確認。

In [5]:
train_df.isnull().sum().sum()

0

In [6]:
test_df.isnull().sum().sum()

0

In [7]:
train_df.dtypes

ip               uint32
app              uint16
device           uint16
os               uint16
channel          uint16
click_time       object
is_attributed     uint8
dtype: object

In [8]:
test_df.dtypes

click_id      uint32
ip            uint32
app           uint16
device        uint16
os            uint16
channel       uint16
click_time    object
dtype: object

In [9]:
train_df.head()

,ip,app,device,os,channel,click_time,is_attributed
0,87540,12,1,13,497,2017-11-07 09:30:38,0
1,105560,25,1,17,259,2017-11-07 13:40:27,0
2,101424,12,1,19,212,2017-11-07 18:05:24,0
3,94584,13,1,13,477,2017-11-07 04:58:08,0
4,68413,12,1,1,178,2017-11-09 09:00:09,0


In [10]:
test_df.head()

,click_id,ip,app,device,os,channel,click_time
0,0,5744,9,1,3,107,2017-11-10 04:00:00
1,1,119901,9,1,3,466,2017-11-10 04:00:00
2,2,72287,21,1,19,128,2017-11-10 04:00:00
3,3,78477,15,1,13,111,2017-11-10 04:00:00
4,4,123080,12,1,13,328,2017-11-10 04:00:00


欠損値はなく、クリック時間以外はint型であることが確認できた。
クリック時間の日時を特徴量として扱うため新しい特徴を作成する。

In [11]:
train_df['hour'] = pd.to_datetime(train_df.click_time).dt.hour.astype('uint8')
train_df['day'] = pd.to_datetime(train_df.click_time).dt.day.astype('uint8')


test_df['hour'] = pd.to_datetime(test_df.click_time).dt.hour.astype('uint8')
test_df['day'] = pd.to_datetime(test_df.click_time).dt.day.astype('uint8')

## 5.モデル作成

今回は多くのカーネルで使用されているlightBGMを学習モデルとして採用する。

In [24]:
target = 'is_attributed'
predictors = ['app', 'device', 'os', 'channel', 'hour', 'day',]
categorical = ['app', 'device', 'os', 'channel', 'hour', 'day']

sub = pd.DataFrame()
sub['click_id'] = test_df['click_id'].astype('int')

xgtrain = lgb.Dataset(train_df[predictors].values, label=train_df[target].values,
                      feature_name=predictors,
                      categorical_feature=categorical,
                      free_raw_data=False
                      )


params = {
    'learning_rate': 0.1,
    # 'is_unbalance': 'true', # replaced with scale_pos_weight argument
    'num_leaves': 15,  # 2^max_depth - 1
    'max_depth': 4,  # -1 means no limit
    'min_child_samples': 100,  # Minimum number of data need in a child(min_data_in_leaf)
    # 'max_bin': 100,  # Number of bucketed bin for feature values
    'subsample': 0.7,  # Subsample ratio of the training instance.
    'subsample_freq': 1,  # frequence of subsample, <=0 means no enable
    'colsample_bytree': 0.9,  # Subsample ratio of columns when constructing each tree.
    'min_child_weight': 0,  # Minimum sum of instance weight(hessian) needed in a child(leaf)
    'scale_pos_weight':99, # because training data is extremely unbalanced
    'metric':'auc'
}

cv_results = lgb.cv(params=params,
                    train_set=xgtrain,
                    nfold=3,
                    num_boost_round=350,
                    early_stopping_rounds=50,
                    verbose_eval=50,
                   )

print(cv_results)



[50]	cv_agg's auc: 0.929195 + 0.0327309
{'auc-mean': [0.8764515747230628, 0.9160956421254424, 0.9336579689708384, 0.9337119057095395, 0.9342754303445582, 0.9341610116698559, 0.9343612400397295, 0.9347418798260044, 0.9382314331903198, 0.9382430708645342, 0.9383269881094035, 0.9379801431987792, 0.9379254289041175, 0.9385182076865788, 0.9387711354155327, 0.9388525004681888, 0.9388178230799893, 0.9387995622444579, 0.9389475379883642, 0.9389360307530508, 0.9387285052565691, 0.9386896353312607, 0.9348282322337093, 0.9348173447527547, 0.9388950056358256, 0.938816291966452, 0.9345779178233032, 0.9345903236749334, 0.9359750206897267, 0.9393528974250883], 'auc-stdv': [0.0471040880099739, 0.013429831690432449, 0.007950485663377322, 0.005601471501026295, 0.008311101840026997, 0.008365375499479312, 0.013071899692138709, 0.012802382825602844, 0.01400217512136441, 0.014051378549869648, 0.013976680514954608, 0.014329491014688141, 0.014348127903302569, 0.018958988315752587, 0.019149349981046403, 0.0190

交差検証で0.928501という数値が出た。
本来ならばこの数値と標準偏差を確認し、そのパラメータで要求を満たすか判断する。

よければそのまま予測値を出力して終了となる。

In [31]:
len_train = len(train_df)
valid_count = 5000
val_df = train_df[(len_train - valid_count):len_train]
train_df = train_df[:(len_train - valid_count)]

xgvalid = lgb.Dataset(val_df[predictors].values, label=val_df[target].values,
                      feature_name=predictors,
                      free_raw_data=False
                      )

evals_results = {}


bst1 = lgb.train(params,
                     xgtrain,
                     valid_sets=[xgtrain, xgvalid],
                     valid_names=['train', 'valid'],
                     evals_result=evals_results,
                     num_boost_round=350,
                     early_stopping_rounds=50,
                     verbose_eval=50
                     )
print("Predicting...")
sub['is_attributed'] = bst1.predict(test_df[predictors])

print("writing...")
sub.to_csv('sub_lgb_cv_001.csv', index=False)
print("done...")

Training until validation scores don't improve for 50 rounds.
[50]	train's auc: 0.97112	valid's auc: 0.997196
[100]	train's auc: 0.975408	valid's auc: 0.99794
[150]	train's auc: 0.974937	valid's auc: 0.998541
Early stopping, best iteration is:
[108]	train's auc: 0.976166	valid's auc: 0.998341
Predicting...
writing...
done...


## まとめ

今回はlightGBMを使って、交差検証を含め一連のフローを完成させることができた。


## 改善点

### パラメータチューニング
- カーネルをそのまま使っているのでグリッドサーチ、ランダムサーチなどの調整するメソッドを使えるようにする。

### 特徴量エンジニアリング
- 今回はできなかったが分析データからIPアドレスごとにカウントした値を追加してモデル学習を行いたい。